In [ ]:
%pip install --no-index "/Volumes/cb_demos/dlt/wheels/brassica-1.0.2-py3-none-any.whl"

In [ ]:
import dlt
from cryptography.fernet import Fernet
import random
import pyspark.sql.functions as F
import brassica

In [ ]:
@dlt.table(name = "sample_trips_bronze")
def sample_trips_bronze():
    key = Fernet.generate_key()
    f = Fernet(key)
    token = str(f.encrypt(b"Guaranteed uniqueness of each deploy/refresh"))
    token = "".join(random.choices(token, k=16))
    return (
        spark.read.table("samples.nyctaxi.trips")
             .withColumn("batch_id", F.lit(token))
    )

In [ ]:
@dlt.table(name = "sample_trips_silver")
def sample_trips_silver():
    col_order = ["batch_id",
                 "pickup_date",
                 "dropoff_date",
                 "trip_distance_mi",
                 "trip_distance_km",
                 "fare_amount",
                 "pickup_zip",
                 "dropoff_zip"]
    
    return (
        dlt.read("sample_trips_bronze")
             .withColumn("pickup_date", F.to_date(F.col("tpep_pickup_datetime")))
             .withColumn("dropoff_date", F.to_date(F.col("tpep_dropoff_datetime")))
             .withColumn("trip_distance_km", F.round(F.col("trip_distance") * 1.60934, 2))
             .withColumnRenamed("trip_distance", "trip_distance_mi")
             .select(col_order)
    )